In [47]:
## Importing Libraries

# Python-based Libraries
from datetime import timedelta, datetime
from pathlib import Path
import os

# Data Analysis Libraries
import numpy as np
import pandas as pd

# Graph plotting libraries

    
# Filtering Data before Plotting


In [63]:
class Drifters:
    
    def __init__(self, name:str):
        self.name = name
        self.data = {}
        self.data_info = {}
        
    def read_data(self, path):
        # Define Possible encodings
        encodings = ["UTF-8", "UTF-16 LE"]

        # Check if path is a filepath or directory path
        if Path(path).is_file(): 
            # Get file name from path
            filename = path.split('/')[-1].split('.')[-2] 
            # Define a variable to track file decoding
            encoding_gotten = False
            for encoding_value in encodings:
                try:
                    data_value = pd.read_csv(path, encoding=encoding_value)
                    # Catch special errors whereby the encoding is wrong but the data is read.
                    if "Unnamed: 1" in data_value.columns: 
                        continue
                    encoding_gotten = True
                    break # Stop trying if the options had worked
                except:
                    pass # Continue trying if the option didn't work

            if encoding_gotten == True:
                data_to_keep = [['DeviceDateTime','Latitude','Longitude'], 
                            ['Position time (UTC)', 'Latitude (°)', 'Longitude (°)'],
                            ]
                for dtk_val in data_to_keep:
                    try:
                        # Select Data to Keep
                        data_value = data_value[dtk_val]
                        # Rename Header Columns
                        data_value.columns = ['DateTime','Latitude','Longitude']
                        # Convert DateTime
                        data_value['DateTime'] = pd.to_datetime(data_value['DateTime'])
                        # Store DF in the object instance 'data'
                        self.data[filename] = data_value
                        
                    except:
                        pass
            else:
                print(f"Encoding error. The encoding for {filename} data should be checked.")

        elif Path(path).is_dir():    
        # 1. Read the files in the given path
            file_names = os.listdir(path)

            # 2. Read each file into pandas dataframe
            for file in file_names:
                filepath = path+'/'+file
                filename = file.split('.')[-2] 
                # Define a variable to track file decoding
                encoding_gotten = False
                for encoding_value in encodings:
                    try:
                        data_value = pd.read_csv(filepath, encoding=encoding_value)
                        # Catch special errors whereby the encoding is wrong but the data is read.
                        if "Unnamed: 1" in data_value.columns: 
                            continue
                        encoding_gotten = True
                        break # Stop trying if the options had worked
                    except:
                        pass # Continue trying if the option didn't work

                # Store the data in the "data" dictionary
                if encoding_gotten == True:
                    data_to_keep = [['DeviceDateTime','Latitude','Longitude'], 
                               ['Position time (UTC)', 'Latitude (°)', 'Longitude (°)'],
                                ]
                    for dtk_val in data_to_keep:
                        try:
                            # Select Data to Keep
                            data_value = data_value[dtk_val]
                            # Rename Header Columns
                            data_value.columns = ['DateTime','Latitude','Longitude']
                            # Convert DateTime
                            data_value['DateTime'] = pd.to_datetime(data_value['DateTime'])
                            # Store DF in the object instance 'data'
                            self.data[filename] = data_value
                        
                        except:
                            pass

                else:
                    print(f"Encoding error. The encoding for {filepath} data should be checked.")
    
    def read_logsheet(self, file_path):
        
        self.logsheet = pd.read_csv(file_path)
        
        self.logsheet.dropna(axis=0, inplace=True)
        
        self.logsheet["DepDateTime"] = self.logsheet.apply(lambda x: str(x["DepDate"]) + " " + str(x["DepTime"]), axis = 1)
        self.logsheet["RecovDateTime"] = self.logsheet.apply(lambda x: str(x["RecovDate"]) + " " + str(x["RecovTime"]), axis = 1)
        
        self.logsheet.drop(columns=["DepDate", "DepTime", "RecovDate", "RecovTime"], inplace=True)
        
        self.logsheet["DepDateTime"] = pd.to_datetime(self.logsheet["DepDateTime"])
        self.logsheet["RecovDateTime"] = pd.to_datetime(self.logsheet["RecovDateTime"])
        
        for key in self.data.keys():
            station = self.logsheet["Station"][self.logsheet["Name"] == float(key)].values
            drog_depth = self.logsheet["DrogDepth"][self.logsheet["Name"] == float(key)].values
            
            
            dep_datetime = self.logsheet["DepDateTime"][self.logsheet["Name"] == float(key)].values
            recov_datetime = self.logsheet["RecovDateTime"][self.logsheet["Name"] == float(key)].values
            
            dep_long = self.logsheet["DepLong"][self.logsheet["Name"] == float(key)].values
            dep_lat = self.logsheet["DepLat"][self.logsheet["Name"] == float(key)].values
            
            recov_long = self.logsheet["RecovLong"][self.logsheet["Name"] == float(key)].values
            recov_lat = self.logsheet["RecovLat"][self.logsheet["Name"] == float(key)].values
            
            
            info = {"Station":station, "DrogDepth":drog_depth, "DepDateTime":dep_datetime, "RecovDateTime":recov_datetime,
                    "DepLong":dep_long, "DepLat":dep_lat, "RecovLong":recov_long, "RecovLat":recov_lat}
            
            self.data_info[key] = info
    
    def time_shift(self, shift_amount:float = 1):
        for key in self.data.keys():
            self.data[key]["DateTime"] = self.data[key]["DateTime"] + timedelta(hours = shift_amount)
    
    def extract_data(self):
        pass
            
    
    def compute_velocity(self):
        R = 6373.0
        for key in self.data.keys():
            data = self.data[key]
            # select columns for latitude, longtitude and time
            df_lat = np.array(data["Latitude"])
            df_long = np.array(data["Longitude"])
            df_time = data["DateTime"]
            
            # compute delta time
            d_time = np.array(df_time[1:]) - df_time[:-1]
            d_time = d_time.apply(lambda dt_i : dt_i.seconds/60.0)
            
            # compute distance from lat,long
            rad_lat = np.radians(df_lat)
            rad_long = np.radians(df_long)
            d_lat = rad_lat[1:] - np.array(rad_lat[:-1])
            d_long = rad_long[1:] - np.array(rad_long[:-1]) 
            
            a = np.sin(d_lat / 2.0)**2 + np.cos(rad_lat[:-1]) * np.cos(rad_lat[1:]) * np.sin(d_long / 2)**2
            c = 2.0 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

            distance = R * c
            velocity = distance*1000.0/60.0/np.array(d_time, dtype=float)
            data["Velocity"] = pd.Series(np.append(0.0, velocity))
    
    def plot_trajectory(self):
        pass
    

In [72]:
# Testing code
D = Drifters("2022")
D.read_data("Data/2022/Day1/drifters/")
D.read_logsheet("Data/2022/Day1/drifters-logsheet.csv")

In [71]:
D.logsheet

,Station,Name,DrogDepth,DepLong,DepLat,RecovLong,RecovLat,DepDateTime,RecovDateTime
0,1,273,1.0,5º 59.928',43º 4.8',5º 59.994,43º 5.342',2022-11-11 10:28:00,2022-11-11 14:05:00
1,1,9666,0.0,5º 59.928',43º 4.8',6º 0.194',43º 5.617',2022-11-11 10:28:00,2022-11-11 14:13:00
2,1,6439,0.6,5º 59.928',43º 4.8',5º 59.399',43º 5.403',2022-11-11 10:28:00,2022-11-11 14:10:00
3,2,274,1.0,5º 59.072',43º 4.695',5º 59.423',43º 4.929',2022-11-11 10:42:00,2022-11-11 13:39:00
4,2,8436,0.0,5º 59.072',43º 4.695',5º 59.282',43º 4.958,2022-11-11 10:42:00,2022-11-11 13:38:00
5,2,3368,0.6,5º 59.072',43º 4.695',5º 59.115',43º 4.993,2022-11-11 10:42:00,2022-11-11 13:35:00
6,3,7230,0.6,5º 58.405',43º 4.644',5º 59.158',43º 4.44',2022-11-11 11:06:00,2022-11-11 13:56:00
7,3,119,0.0,5º 58.405',43º 4.644',5º 59.192',43º 4.552',2022-11-11 11:06:00,2022-11-11 13:57:00
8,3,277,1.0,5º 58.405',43º 4.644',5º 59.158',43º 4.44',2022-11-11 11:06:00,2022-11-11 13:56:00
9,4,2052,0.6,6º 0.605',42º 59.533',-,-,2022-11-11 11:43:00,2022-11-11 12:14:00
